### 터미널에서 Kubectl 설치 필요
```
sudo apt-get update && sudo apt-get install -y apt-transport-https gnupg2
curl -s https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
echo "deb https://apt.kubernetes.io/ kubernetes-xenial main" | sudo tee -a /etc/apt/sources.list.d/kubernetes.list
sudo apt-get update
sudo apt-get install -y kubectl=1.16.9-00
```

### 전체 파드 조회가 필요하므로 .kube/config 설정 필요
```
vi .kube/config
// cto config 설정 //
```

In [66]:
import os
import requests
import json
import pandas as pd
import subprocess

from spellchecker import SpellChecker

In [67]:
prom_query_url = 'http://13.125.63.249:30915/api/v1/query?query='

In [68]:
def call_data(url, query):
    r = requests.get(url + query)
    return r

In [69]:
url_prom = 'http://13.125.63.249:30915/api/v1/query?query='
query = 'ALERTS{alertname="KubeContainerWaiting"}'
json.loads(call_data(url_prom, query).text)


{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {'__name__': 'ALERTS',
     'alertname': 'KubeContainerWaiting',
     'alertstate': 'pending',
     'container': 'mysqll',
     'namespace': 'test-alerts',
     'pod': 'test-mysql-5f88855958-446vq',
     'severity': 'warning'},
    'value': [1600678749.289, '1']},
   {'metric': {'__name__': 'ALERTS',
     'alertname': 'KubeContainerWaiting',
     'alertstate': 'pending',
     'container': 'test-nginx',
     'namespace': 'test-alerts',
     'pod': 'test-nginx',
     'severity': 'warning'},
    'value': [1600678749.289, '1']}]}}

In [70]:
def kube_container_waiting():
    ## Alerts case : KubeContainerWaiting
    alert_query = 'ALERTS{alertname="KubeContainerWaiting"}'
    alert_get = call_data(prom_query_url, alert_query)
    alert_message = json.loads(alert_get.text)
    
    fix_list = []
    for item in alert_message['data']['result']:
        #print(item)
        check_list = {'namespace':item['metric']['namespace'], 'pod':item['metric']['pod']}
        #check_list = {item['metric']['namespace'], item['metric']['pod']}
        print(check_list)        
        fix_list.append(check_list)     
    
    return(fix_list) 

In [71]:
def kubectl_jsonpath():
    pending_data = kube_container_waiting()
    cnt = len(pending_data)    
    df_data = pd.DataFrame(pending_data)
    #print(df_data)
    
    kubectl_result = []
    for i in range(cnt):
        namespace = df_data.namespace[i]
        pod = df_data.pod[i]
        
        # !kubectl -n aiops get pods test-pending -o jsonpath='{.status.containerStatuses[*].state.waiting.message}' | awk -F"\"" '{print $2}'
        cli = "kubectl -n " + namespace + " get pods " + pod + \
              " -o jsonpath='{.status.containerStatuses[*].state.waiting.message}'" + \
              " | awk -F'\"' '{print $2}'"
        
        kubectl_command_line = subprocess.check_output(cli, shell=True,  encoding='utf-8')
        print("Pod 이름 : " + pod + " 현재 적용된 이미지 : " + kubectl_command_line)
        tmp_string = namespace + " " + pod + " " + kubectl_command_line
        
        ### sort :: namespace pod result        
        kubectl_result.append(tmp_string)
        
    return(kubectl_result)
    


In [72]:
def recommand_image():
    pending_images = kubectl_jsonpath()
    cnt = len(pending_images)
    spell = SpellChecker()
    spell.word_frequency.load_text_file('./words.txt')
    
    #result = []
    for i in range(cnt):
        namespace = pending_images[i].split(' ')[0]
        pod = pending_images[i].split(' ')[1]   
        m_image = pending_images[i].split(' ')[2]
        
        #misspelled = spell.unknown([m_image])
        tmp_recommend = spell.correction(m_image)
        tmp_string = "namespace: " + namespace \
                   + " , pod: " + pod \
                   + "\n" + "   =====> Please Check your Pull Image. May be : " + tmp_recommend 
        result = tmp_string
        print(result)
        
    #return(result)
    

In [73]:
if __name__=="__main__":
    #print(kube_container_waiting())
    #print(kubectl_jsonpath())
    print(recommand_image())
    

{'namespace': 'test-alerts', 'pod': 'test-mysql-5f88855958-446vq'}
{'namespace': 'test-alerts', 'pod': 'test-nginx'}
Pod 이름 : test-mysql-5f88855958-446vq 현재 적용된 이미지 : mysqll

Pod 이름 : test-nginx 현재 적용된 이미지 : niginx

namespace: test-alerts , pod: test-mysql-5f88855958-446vq
   =====> Please Check your Pull Image. May be : mysql
namespace: test-alerts , pod: test-nginx
   =====> Please Check your Pull Image. May be : nginx
None
